In [1]:
import tensorflow as tf # TODO

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
import numpy
import matplotlib.pyplot as plt

In [ ]:
# Parameters
learning_rate = 0.005
training_epochs = 50
batch_size = 50
display_step = 5

In [4]:
def mackey_glass(t, t25):
    return 0.9 * t + 0.2*t25/(1+t25**10)
fdata = [1.5]
for i in range(1525):
    fdata.append(mackey_glass(fdata[i], fdata[i - 25] if i > 25 else 0))

In [5]:
def create_mg_series(i, data):
    return numpy.asarray([data[i-20], data[i-15], data[i-10], data[i-5], data[i]])

train_X = numpy.asarray(list(map(lambda x: create_mg_series(x, fdata), range(301, 1300))))
train_Y = numpy.asarray(list(map(lambda x: fdata[x+5], range(301, 1300))))
test_X = numpy.asarray(list(map(lambda x: create_mg_series(x, fdata), range(1301, 1500))))
test_Y = numpy.asarray(list(map(lambda x: fdata[x+5], range(1301, 1500))))

In [24]:
# X Y
X = tf.placeholder(tf.float32, [1,5], name='x')  # 5 inputs
Y = tf.placeholder(tf.float32,name='y')  # 1 output

# hidden layer 1
W1 = tf.Variable(tf.truncated_normal([5, 8], stddev=0.03), name='W1')
b1 = tf.Variable(tf.truncated_normal([8]), name='b1')

# hidden layer 2
W2 = tf.Variable(tf.truncated_normal([8, 5], stddev=0.03), name='W2')
b2 = tf.Variable(tf.truncated_normal([5]), name='b2')

In [25]:
# output hidden layer 1
hidden_out = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))

# total output
y_ = tf.nn.relu(tf.add(tf.matmul(hidden_out, W2), b2))

In [26]:
mse = tf.losses.mean_squared_error(Y, y_)

In [27]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(mse)

In [28]:
# initialize variables
init_op = tf.global_variables_initializer()

In [29]:
# accuracy for the test set
accuracy = tf.reduce_mean(tf.square(tf.subtract(Y, y_))) # or could use tf.losses.mean_squared_error

In [ ]:
with tf.Session() as sess:
    sess.run(init_op)
    batches = int(len(train_Y) / batch_size) 
    # Fit all training data
    for epoch in range(training_epochs):
        avg_cost = 0
        for (x, y) in zip(train_X, train_Y):
            _, cost = sess.run([optimizer, mse], feed_dict={X: x.reshape(1,5), Y: y.reshape(1,1)})
            avg_cost += cost / training_epochs

        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            # c = sess.run(mse, feed_dict={X: train_X, Y:train_Y})
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    # print("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')

    print("Testing... (Mean square loss Comparison)")
    testing_cost = 0
    for epoch in range(training_epochs):
        for (x, y) in zip(test_X, train_Y):
            cost = sess.run(
                tf.reduce_sum(tf.pow(y_ - Y, 2)) / (2 * test_X.shape[0]),
                feed_dict={X: x.reshape(1,5), Y: y.reshape(1,1)})  # same function as cost above
            testing_cost += cost / training_epochs

    print("Testing cost=", testing_cost)
    print("Absolute mean square loss difference:", abs(
        training_cost - testing_cost))

Epoch: 0005 cost= 11.410031303
Epoch: 0010 cost= 11.385859016
Epoch: 0015 cost= 11.293049645
Epoch: 0020 cost= 11.108462305
Epoch: 0025 cost= 11.023003088
Epoch: 0030 cost= 10.999154258
Epoch: 0035 cost= 10.983920994
Epoch: 0040 cost= 10.971348844
Epoch: 0045 cost= 10.960514760
Epoch: 0050 cost= 10.951137966
Testing... (Mean square loss Comparison)


In [ ]:
X

In [ ]:
Y

In [ ]:
x

In [ ]:
y